In [4]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

df = pd.read_csv("../data/processed_chunks.csv")
model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = model.encode(df['review_chunk'].tolist(), show_progress_bar=True)


Batches: 100%|██████████| 641/641 [05:28<00:00,  1.95it/s]


In [5]:
from qdrant_client import QdrantClient

client = QdrantClient(host="localhost", port=6333)

print("Connected to Qdrant!")

Connected to Qdrant!


In [6]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

client = QdrantClient(host="localhost", port=6333, timeout=60.0)  

collection_name = "hotel_reviews"

client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=len(embeddings[0]), distance=Distance.COSINE)
)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_16676\999103623.py:8: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [7]:
points = []

for idx, (embedding, row) in enumerate(zip(embeddings, df.itertuples())):
    points.append(PointStruct(
        id=idx,
        vector=embedding,
        payload={
            "original_index": int(row.original_index),
            "rating": int(row.rating),
            "text": row.review_chunk
        }
    ))

"""
UnexpectedResponse: Unexpected Response: 400 (Bad Request)
Raw response content:
b'{"status":{"error":"Payload error: JSON payload (180458192 bytes) is larger than allowed (limit: 33554432 bytes)."},"time":0.0}'
"""

batch_size = 500

for i in range(0, len(points), batch_size):
    batch = points[i:i+batch_size]
    client.upsert(collection_name=collection_name, points=batch)
    print(f"Uploaded batch {i} to {i + len(batch)}")



Uploaded batch 0 to 500
Uploaded batch 500 to 1000
Uploaded batch 1000 to 1500
Uploaded batch 1500 to 2000
Uploaded batch 2000 to 2500
Uploaded batch 2500 to 3000
Uploaded batch 3000 to 3500
Uploaded batch 3500 to 4000
Uploaded batch 4000 to 4500
Uploaded batch 4500 to 5000
Uploaded batch 5000 to 5500
Uploaded batch 5500 to 6000
Uploaded batch 6000 to 6500
Uploaded batch 6500 to 7000
Uploaded batch 7000 to 7500
Uploaded batch 7500 to 8000
Uploaded batch 8000 to 8500
Uploaded batch 8500 to 9000
Uploaded batch 9000 to 9500
Uploaded batch 9500 to 10000
Uploaded batch 10000 to 10500
Uploaded batch 10500 to 11000
Uploaded batch 11000 to 11500
Uploaded batch 11500 to 12000
Uploaded batch 12000 to 12500
Uploaded batch 12500 to 13000
Uploaded batch 13000 to 13500
Uploaded batch 13500 to 14000
Uploaded batch 14000 to 14500
Uploaded batch 14500 to 15000
Uploaded batch 15000 to 15500
Uploaded batch 15500 to 16000
Uploaded batch 16000 to 16500
Uploaded batch 16500 to 17000
Uploaded batch 17000 to 

In [8]:
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer

# Load the embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Initialize Qdrant client
client = QdrantClient(host="localhost", port=6333, timeout=60.0)

# Define collection name
collection_name = "hotel_reviews"

# Query example
query = "best hotel near downtown with free wifi"
query_vec = model.encode(query)

# Search using query_points instead of deprecated search
results = client.query_points(
    collection_name=collection_name,
    query=query_vec,
    limit=5
)

# Print results
for res in results.points:
    print(f"Score: {res.score:.2f}")
    print(res.payload['text'])
    print('-' * 50)

Score: 0.69
fantastic hotel fantastic hotel staff helpful friendlyrooms quite dark nice decoration bathroom fantasticthere free wifi room ethernet wired connection room reception happy land cable free course location excellent close major metro station starbucks minute walkwe stayed night loved minute highly recommended
--------------------------------------------------
Score: 0.68
great location pleasantly surprised hotel ideal downtown despite review felt safe areawe use valet parking option nightrooms basic finebreakfast included aaa rate great everyday served sort cafe lobby
--------------------------------------------------
Score: 0.66
good location value money great location great value price paid good customer service lacking thing free wireless access room wireless accessible hotel lobby convenient heating control flexible sleep window open hot night overall good hotel short stay
--------------------------------------------------
Score: 0.65
spectacular service reception roomma

In [ ]:
# from crewai import Agent
# from qdrant_client import QdrantClient
# from sentence_transformers import SentenceTransformer
# import openai

# openai.api_key = "your_openai_api_key"

# class RetrieverAgent(Agent):
#     def __init__(self, model: SentenceTransformer, client: QdrantClient, collection_name: str):
#         super().__init__(
#             role="Retriever Agent",
#             goal="Search the knowledge base for relevant travel information",
#             backstory="This agent is responsible for retrieving relevant hotel or travel reviews from the vector database."
#         )
#         self.model = model
#         self.client = client
#         self.collection_name = collection_name

#     def run(self, query: str):
#         query_vec = self.model.encode(query)
#         results = self.client.query_points(
#             collection_name=self.collection_name,
#             query=query_vec,
#             limit=5
#         )
#         texts = [res.payload['text'] for res in results.points]
#         return texts

# class SummarizerAgent(Agent):
#     def __init__(self):
#         super().__init__(
#             role="Summarizer Agent",
#             goal="Summarize retrieved travel data and remove redundant information",
#             backstory="This agent combines and summarizes the retrieved content to prepare it for the final response."
#         )

#     def run(self, texts: list[str]):
#         combined_text = " ".join(texts)
#         prompt = f"""
#         You are a travel assistant. Summarize the following hotel reviews into a concise summary (max 200 words), removing redundant information and focusing on key points like location, amenities, and service quality:
#         {combined_text[:2000]}  # Limit input to avoid token overflow
#         """
#         response = openai.chat.completions.create(
#             model="gpt-3.5-turbo",
#             messages=[
#                 {"role": "system", "content": "You are a helpful travel assistant."},
#                 {"role": "user", "content": prompt}
#             ],
#             temperature=0.5
#         )
#         return response.choices[0].message.content

# class ComposerAgent(Agent):
#     def __init__(self):
#         super().__init__(
#             role="Composer Agent",
#             goal="Generate user-facing responses using LLMs",
#             backstory="This agent crafts the final travel recommendation using GPT based on the summarized input."
#         )

#     def run(self, summary: str):
#         prompt = f"""
#         You are a helpful travel assistant.

#         Here is a summary of reviews:
#         {summary}

#         Please write a hotel recommendation or short itinerary in a friendly, concise tone.
#         """
#         response = openai.chat.completions.create(
#             model="gpt-3.5-turbo",
#             messages=[
#                 {"role": "system", "content": "You are a helpful travel assistant."},
#                 {"role": "user", "content": prompt}
#             ],
#             temperature=0.7
#         )
#         return response.choices[0].message.content

In [29]:
# from qdrant_client import QdrantClient
# from sentence_transformers import SentenceTransformer
# from crewai_agents import RetrieverAgent, SummarizerAgent, ComposerAgent

# # Initialize dependencies
# model = SentenceTransformer('all-MiniLM-L6-v2')
# client = QdrantClient(host="localhost", port=6333, timeout=60.0)
# collection_name = "hotel_reviews"

# # Create agent instances
# retriever = RetrieverAgent(model=model, client=client, collection_name=collection_name)
# summarizer = SummarizerAgent()
# composer = ComposerAgent()

# def agentic_pipeline(user_query: str):
#     retrieved_texts = retriever.run(user_query)
#     summary = summarizer.run(retrieved_texts)
#     final_response = composer.run(summary)
#     return final_response

# # Test the pipeline
# if __name__ == "__main__":
#     query = "best hotel near downtown with free wifi"
#     response = agentic_pipeline(query)
#     print(response)

In [ ]:
# from sentence_transformers import SentenceTransformer
# from pydantic import BaseModel

# model = SentenceTransformer('all-MiniLM-L6-v2') 

# class RetrieverAgent(BaseModel):
#     role: str = "Retriever"
#     goal: str = "Retrieve relevant hotel reviews"
#     backstory: str = "Searches hotel review database"

#     def run(self, query: str):
#         query_vec = model.encode(query) 
#         results = client.search(
#             collection_name=collection_name,
#             query_vector=query_vec,
#             limit=5
#         )
#         texts = [res.payload['text'] for res in results]
#         return "\n".join(texts)